In [224]:
import pm4py
ocel = pm4py.read_ocel2_sqlite("/home/grkmr/data/ocels/ContainerLogistics.sqlite")

/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:75: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_23_foreign_key_event_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/util/dataframe_utils.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pandas_utils.dataframe_column_string_to_datetime(df[col], format=timest_format, utc=True)
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/u

In [235]:
from typing import Literal, Optional, cast
from dataclasses import dataclass

import pandas as pd
from pandas.core.series import Series
from pm4py.objects.ocel.obj import OCEL
from pydantic import BaseModel


from typing import Literal, Optional, cast

from pandas.core.series import Series

@dataclass
class FilterResult:
    events: Optional[pd.Series] = None
    objects: Optional[pd.Series] = None

    def and_merge(self, other: "FilterResult") -> "FilterResult":
        def _and(a, b):
            if a is not None and b is not None:
                return a & b
            elif a is not None:
                return a
            elif b is not None:
                return b
            else:
                return None

        return FilterResult(
            events=_and(self.events, other.events),
            objects=_and(self.objects, other.objects),
        )


def getO2OWithTypes(ocel, direction: Literal["source", "target"] = "source"):
    o2o_with_types = pd.merge(
        ocel.o2o,
        ocel.objects[[ocel.object_id_column, ocel.object_type_column]],
        how="left",
    )
    o2o_with_types = pd.merge(
        o2o_with_types,
        ocel.objects[[ocel.object_id_column, ocel.object_type_column]],
        how="left",
        left_on=f"{ocel.object_id_column}_2",
        right_on=ocel.object_id_column,
        suffixes=["", "_new"],  # type:ignore
    )
    o2o_with_types = o2o_with_types[
        [
            ocel.object_id_column,
            f"{ocel.object_id_column}_2",
            ocel.qualifier,
            ocel.object_type_column,
            f"{ocel.object_type_column}_new",
        ]
    ]
    rename_map = {
        ocel.object_id_column: direction,
        f"{ocel.object_id_column}_2": "target" if direction == "source" else "source",
        ocel.qualifier: "qualifier",
        ocel.object_type_column: f"{direction}_type",
        f"{ocel.object_type_column}_new": f"{'target' if direction == 'source' else 'source'}_type",
    }
    return o2o_with_types.rename(columns=rename_map)  # type:ignore



class RelationCountFilterConfig(BaseModel):
    source: str
    target: str
    mode:Optional[Literal["include", "exclude"]] = "include"
    range: tuple[Optional[int], Optional[int]]
    qualifier: Optional[str] = None
    direction: Optional[Literal["source", "target"]] = "source"

def filter_by_relation_counts(
    ocel,
    relation_table: pd.DataFrame,
    source_id_column:str,
    source_column: str,
    target_column: str,
    qualifier_column:str,
    entity_id_column: str,
    entity_type_column: str,
    source_df: pd.DataFrame,
    config: RelationCountFilterConfig,
) -> FilterResult:
    # Get o2o with types if not provided
    relation_table = relation_table[(relation_table[source_column] == config.source) & (relation_table[target_column]==config.target)]
    
    if(config.qualifier is not None):
        relation_table = relation_table[relation_table[qualifier_column] == config.qualifier]

    # Count how many times each target appears
    entity_counts = cast(Series, relation_table.groupby(source_id_column).size()).reset_index(
        name="entity_count"
    )

    min_count, max_count = config.range

    if min_count is not None:
        entity_counts = entity_counts[entity_counts["entity_count"] >= min_count]
    if max_count is not None:
        entity_counts = entity_counts[entity_counts["entity_count"] <= max_count]
    entity_counts = entity_counts[source_id_column]

    
    if min_count == 0:
        merged = pd.merge(
            source_df[source_df[entity_type_column] == config.source], 
            relation_table, 
            left_on=entity_id_column, 
            right_on=source_id_column, 
            how="left",
            indicator=True
        )
        entities_with_no_relations = merged.loc[
            merged["_merge"] == "left_only", entity_id_column
        ]
        
        entity_counts = pd.concat([entity_counts, entities_with_no_relations])

    # Mask for non-target-type objects (always kept)
    is_not_target_type = source_df[entity_type_column] != config.source

    # Mask for objects meeting the relation count condition
    is_in_filtered_ids = source_df[entity_id_column].isin(entity_counts)

    # Invert if in exclude mode
    if config.mode == "exclude":
        is_in_filtered_ids = ~is_in_filtered_ids

    # Final mask: keep non-target-type or qualifying objects
    final_mask = cast(Series, is_not_target_type | is_in_filtered_ids)
    
    return FilterResult(objects=final_mask)



In [236]:
o2oTable = getO2OWithTypes(ocel)

In [237]:
result = filter_by_relation_counts(
    ocel=ocel,
    relation_table=o2oTable,
    source_column="source_type",
    target_column="target_type",
    source_id_column="source",
    qualifier_column="qualifier",
    entity_id_column=ocel.object_id_column,
    entity_type_column=ocel.object_type_column,
    source_df=ocel.objects,
    config=RelationCountFilterConfig(
        source="Transport Document", 
        target="Vehicle",
        range=[0,1],
        qualifier="Ersatz VH for TD" 
    )
)

      source target  qualifier     source_type         target_type
0        co1    td6  TD for CO  Customer Order  Transport Document
1        co2    td1  TD for CO  Customer Order  Transport Document
2        co3    td7  TD for CO  Customer Order  Transport Document
3        co4    td9  TD for CO  Customer Order  Transport Document
4        co5    td5  TD for CO  Customer Order  Transport Document
...      ...    ...        ...             ...                 ...
3365  cr2011  td590  CR for TD       Container  Transport Document
3366  cr1999  td586  CR for TD       Container  Transport Document
3367  cr1989  td583  CR for TD       Container  Transport Document
3368  cr1993  td584  CR for TD       Container  Transport Document
3369  cr2019  td592  CR for TD       Container  Transport Document

[3370 rows x 5 columns]


In [240]:
class E2OCountFilterConfig(BaseModel):
    type: Literal["e2o_count"]
    source: str
    target: str
    range: tuple[Optional[int], Optional[int]]
    direction: Literal["source", "target"] = "source"


def filter_by_e2o_count(ocel:pm4py.OCEL, config:E2OCountFilterConfig):
    source_column = ocel.event_activity if config.direction == "source" else ocel.object_type_column
    target_column = ocel.object_type_column if config.direction == "source" else ocel.event_activity
    source_id_column = ocel.event_id_column if config.direction == "source" else ocel.object_id_column
    entity_id_column = source_id_column
    qualifier_column = ocel.qualifier
    entity_type_column = source_column
    return filter_by_relation_counts(
        ocel=ocel,
        relation_table=ocel.relations,
        source_column=source_column,
        target_column=target_column,
        source_id_column=source_id_column,
        qualifier_column=qualifier_column,
        entity_id_column=entity_id_column,
        entity_type_column=entity_type_column,
        source_df=ocel.events if config.direction =="source" else ocel.objects,
        config=RelationCountFilterConfig(**config.model_dump())
    )

In [247]:
result = filter_by_e2o_count(
    ocel, 
    E2OCountFilterConfig(
        type="e2o_count",
        source="Book Vehicles",
        target="Vehicle",
        range=(0, 2),
        direction="source"
    )
)

                 ocel:eid ocel:oid     ocel:qualifier  \
0                 reg_co1      co1      registered CO   
1                 reg_co2      co2      registered CO   
600            create_td1      co2  TD created for CO   
601            create_td1      td1         created TD   
1788        book_vehs_td1      td1  VHs booked for TD   
...                   ...      ...                ...   
65487       to_bay_cr1935   cr1935  CR brought to bay   
65489       to_bay_cr1937   cr1937  CR brought to bay   
49963  load_truck_hu10557   cr2018           CR laded   
71386     load_veh_cr1939   cr1939          CR loaded   
49966  load_truck_hu10544   cr1980           CR laded   

                   ocel:activity            ocel:timestamp           ocel:type  
0        Register Customer Order 2023-05-22 11:54:42+00:00      Customer Order  
1        Register Customer Order 2023-05-22 18:33:30+00:00      Customer Order  
600    Create Transport Document 2023-05-23 08:22:17+00:00      Customer

In [248]:
ocel.events[result.objects]

,ocel:eid,ocel:timestamp,ocel:activity
0,reg_co1,2023-05-22 11:54:42+00:00,Register Customer Order
1,reg_co2,2023-05-22 18:33:30+00:00,Register Customer Order
0,create_td1,2023-05-23 08:22:17+00:00,Create Transport Document
0,book_vehs_td1,2023-05-23 10:05:16+00:00,Book Vehicles
2,reg_co3,2023-05-23 12:14:57+00:00,Register Customer Order
...,...,...,...
1968,to_bay_cr1937,2024-08-21 14:54:34+00:00,Bring to Loading Bay
10551,load_truck_hu10557,2024-08-21 14:57:48+00:00,Load Truck
10552,collect_hu10544,2024-08-21 14:57:48+00:00,Collect Goods
1965,load_veh_cr1939,2024-08-21 16:16:39+00:00,Load to Vehicle


In [249]:
ocel.events

,ocel:eid,ocel:timestamp,ocel:activity
0,reg_co1,2023-05-22 11:54:42+00:00,Register Customer Order
1,reg_co2,2023-05-22 18:33:30+00:00,Register Customer Order
0,create_td1,2023-05-23 08:22:17+00:00,Create Transport Document
0,book_vehs_td1,2023-05-23 10:05:16+00:00,Book Vehicles
2,reg_co3,2023-05-23 12:14:57+00:00,Register Customer Order
...,...,...,...
1968,to_bay_cr1937,2024-08-21 14:54:34+00:00,Bring to Loading Bay
10551,load_truck_hu10557,2024-08-21 14:57:48+00:00,Load Truck
10552,collect_hu10544,2024-08-21 14:57:48+00:00,Collect Goods
1965,load_veh_cr1939,2024-08-21 16:16:39+00:00,Load to Vehicle
